In [579]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler
import time
import matplotlib.pyplot as plt
import sklearn.preprocessing as preprocessing
import sklearn.metrics
from sklearn.model_selection import train_test_split
import math
from torch.utils.data import DataLoader, TensorDataset
from torch import Tensor
from sklearn.utils import resample

In [589]:
x_train = pd.read_csv('x_train_full.csv')
x_val = pd.read_csv('x_val_full.csv')
x_test = pd.read_csv('x_test_full.csv')
y_train = pd.read_csv('./Data with correct player_atts/y_train.csv')
y_val = pd.read_csv('./Data with correct player_atts/y_val.csv')
y_test = pd.read_csv('./Data with correct player_atts/y_test.csv')

x_full = pd.concat([x_train,x_val,x_test], axis = 0)
y_full = pd.concat([y_train,y_val,y_test], axis = 0)

match = pd.read_csv('./Data/Match.csv')

In [590]:
x_full = x_full.merge(match[['match_api_id','league_id']], how = 'left', on ='match_api_id')
y_full = y_full.merge(match[['match_api_id','league_id']], how = 'left', on = 'match_api_id')

assert (x_full.match_api_id == y_full.match_api_id).sum() == len(x_full)

## Data Processing

In [591]:
label_dict = {1:0, 0:1, -1:2 }
y_full.loc[:,'target'] = y_full['result'].map(label_dict)

In [680]:
x_pl = x_full[x_full['league_id'].isin([1729,21518,7809,10257,4769])].drop(columns ='league_id')
y_pl = y_full[y_full['league_id'].isin([1729,21518,7809,10257,4769])].drop(columns ='league_id')

x_train, x_test, y_train, y_test = train_test_split(x_pl,y_pl, test_size = 0.1)
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train, test_size = 0.12)

x_train.shape

(9614, 35)

In [681]:
## creating different training set for ensemble learning

x1, y1 = resample(x_train, y_train, random_state=0)
x2, y2 = resample(x_train, y_train, random_state=1)
x3, y3 = resample(x_train, y_train, random_state=2)

In [597]:
# StandardScaler = preprocessing.StandardScaler().fit(x_train.iloc[:,1:])

# x_train = pd.DataFrame(StandardScaler.transform(x_train.iloc[:,1:]), index = x_train['match_api_id'], columns = x_train.iloc[:,1:].columns)
# x_val = pd.DataFrame(StandardScaler.transform(x_val.iloc[:,1:]), index = x_val['match_api_id'], columns = x_val.iloc[:,1:].columns)
# x_test = pd.DataFrame(min_max_scaler.transform(x_test.iloc[:,1:]), index = x_test['match_api_id'], columns = x_test.iloc[:,1:].columns)

In [682]:
y_train['result'].value_counts(normalize=True)

 1    0.458394
-1    0.283025
 0    0.258581
Name: result, dtype: float64

## Training

In [689]:
# Build Network
input_size = len(x_train.drop(columns ='match_api_id').columns)
hidden_sizes = [128,512,512,128,32,16]
output_size = 3
prob = 0.7
# Build a feed-forward network
LinearNN1 = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                          nn.Dropout(p=prob),
                         nn.BatchNorm1d(hidden_sizes[0]),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                          nn.Dropout(p=prob),
                         nn.BatchNorm1d(hidden_sizes[1]),
                      nn.Linear(hidden_sizes[1], hidden_sizes[2]),
                        nn.ReLU(),
                          nn.Dropout(p=prob),
                         nn.BatchNorm1d(hidden_sizes[2]),
                        nn.Linear(hidden_sizes[2], hidden_sizes[3]),
                        nn.ReLU(),
                          nn.Dropout(p=prob),
                         nn.BatchNorm1d(hidden_sizes[3]),
                        nn.Linear(hidden_sizes[3], hidden_sizes[4]),
                        nn.ReLU(),
                         nn.Dropout(p=prob),
                         nn.BatchNorm1d(hidden_sizes[4]),
                        nn.Linear(hidden_sizes[4], hidden_sizes[5]),
                        nn.ReLU(),
                         nn.Dropout(p=prob),
                         nn.BatchNorm1d(hidden_sizes[5]),
                         nn.Linear(hidden_sizes[5], output_size),
                      nn.Softmax(dim = 1)
                        )



In [690]:
# Build Network
input_size = len(x_train.drop(columns ='match_api_id').columns)
hidden_sizes = [64,128,256,128,32,16]
output_size = 3
prob = 0.7
# Build a feed-forward network
LinearNN2 = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                          nn.Dropout(p=prob),
                         nn.BatchNorm1d(hidden_sizes[0]),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                          nn.Dropout(p=prob),
                         nn.BatchNorm1d(hidden_sizes[1]),
                      nn.Linear(hidden_sizes[1], hidden_sizes[2]),
                        nn.ReLU(),
                          nn.Dropout(p=prob),
                         nn.BatchNorm1d(hidden_sizes[2]),
                        nn.Linear(hidden_sizes[2], hidden_sizes[3]),
                        nn.ReLU(),
                          nn.Dropout(p=prob),
                         nn.BatchNorm1d(hidden_sizes[3]),
                        nn.Linear(hidden_sizes[3], hidden_sizes[4]),
                        nn.ReLU(),
                         nn.Dropout(p=prob),
                         nn.BatchNorm1d(hidden_sizes[4]),
                        nn.Linear(hidden_sizes[4], hidden_sizes[5]),
                        nn.ReLU(),
                         nn.Dropout(p=prob),
                         nn.BatchNorm1d(hidden_sizes[5]),
                         nn.Linear(hidden_sizes[5], output_size),
                      nn.Softmax(dim = 1)
                        )



In [691]:
# Build Network
input_size = len(x_train.drop(columns ='match_api_id').columns)
hidden_sizes = [64,128,256,128,32,16,8]
output_size = 3
prob = 0.7
# Build a feed-forward network
LinearNN3 = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.Tanh(),
                          nn.Dropout(p=prob),
                         nn.BatchNorm1d(hidden_sizes[0]),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.Tanh(),
                          nn.Dropout(p=prob),
                         nn.BatchNorm1d(hidden_sizes[1]),
                      nn.Linear(hidden_sizes[1], hidden_sizes[2]),
                        nn.Tanh(),
                          nn.Dropout(p=prob),
                         nn.BatchNorm1d(hidden_sizes[2]),
                        nn.Linear(hidden_sizes[2], hidden_sizes[3]),
                        nn.Tanh(),
                          nn.Dropout(p=prob),
                         nn.BatchNorm1d(hidden_sizes[3]),
                        nn.Linear(hidden_sizes[3], hidden_sizes[4]),
                        nn.ReLU(),
                         nn.Dropout(p=prob),
                         nn.BatchNorm1d(hidden_sizes[4]),
                        nn.Linear(hidden_sizes[4], hidden_sizes[5]),
                        nn.ReLU(),
                         nn.Dropout(p=prob),
                         nn.BatchNorm1d(hidden_sizes[5]),
                            nn.Linear(hidden_sizes[5], hidden_sizes[6]),
                        nn.ReLU(),
                         nn.Dropout(p=prob),
                         nn.BatchNorm1d(hidden_sizes[6]),
                         nn.Linear(hidden_sizes[6], output_size),
                      nn.Softmax(dim = 1)
                        )



In [567]:
# Hyperparameters

batch_size = 512 #len(x_train)
epochs = 20
learning_rate = 0.001
loss = 0
losses = []
losses_val = []
counter = 0


dataset = TensorDataset( Tensor(x_train.values), torch.Tensor(y_train['target'].values) )
train_loader = DataLoader(dataset, batch_size = batch_size, shuffle=False)

optimizer = torch.optim.Adam(LinearNN.parameters(), lr=learning_rate, betas=(0.9,0.999))

scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 60 , gamma = 0.1)
mse = torch.nn.MSELoss()
cross = nn.CrossEntropyLoss()

In [ ]:
#Train Model

t = time.time()
for epoch in range(epochs):
    for x, y in iter(train_loader):
        LinearNN.train()
        LinearNN.zero_grad()
        
        y_pred = LinearNN(x)
        y_val_pred = LinearNN(Tensor(x_val.values))

        loss = cross(y_pred, y.long())
        loss_val = cross(y_val_pred, Tensor(y_val['target'].values).long())

        if counter % 500 ==0:
            print('Loss after iteration {}: {}'.format(counter, loss.item()))
        
        losses.append(loss.item())
        losses_val.append(loss_val.item())
        
        loss.backward()
        optimizer.step()
        
        counter+=1 
    counter+=1
    scheduler.step()
time.time()-t        
        
        
print('Elapsed time: {} s'.format(time.time()-t))    
print(loss.item())  
plt.plot(range(len(losses)), losses, label = 'train')
plt.plot(range(len(losses)), losses_val, label = 'val')
plt.legend(loc="lower left")
plt.show()

In [ ]:
##Eval 

LinearNN.eval()
with torch.no_grad():
    val_pred = LinearNN(Tensor(x_val.values))
    train_pred = LinearNN(Tensor(x_train.values))
y_val_pred = pd.Series(val_pred.max(1).indices).map({0:1,1:0,2:-1})
y_train_pred = pd.Series(train_pred.max(1).indices).map({0:1,1:0,2:-1})

print(sklearn.metrics.accuracy_score(y_train_pred, y_train['result']))
print(sklearn.metrics.accuracy_score(y_val_pred, y_val['result']))

In [555]:
y_val_pred.value_counts()

 1    1008
-1     457
dtype: int64

In [ ]:
##Test 

LinearNN.eval()
with torch.no_grad():
    val_pred = LinearNN(Tensor(x_test.values))
    
y_val_pred = pd.Series(val_pred.max(1).indices).map({0:1,1:0,2:-1})
sklearn.metrics.accuracy_score(y_val_pred, y_test['result'])

In [275]:
y_val_pred.value_counts()

 1    797
-1    296
dtype: int64

## Ensemble Learning

In [686]:
def train(model, X_train,Y_train, X_val, Y_val,  epochs, batch_size, learning_rate):
    StandardScaler = preprocessing.StandardScaler().fit(X_train.iloc[:,1:])
    
    X_train = pd.DataFrame(StandardScaler.transform(X_train.iloc[:,1:]), index = X_train['match_api_id'], columns = X_train.iloc[:,1:].columns)
    X_val = pd.DataFrame(StandardScaler.transform(X_val.iloc[:,1:]), index = X_val['match_api_id'], columns = X_val.iloc[:,1:].columns)
    
    # Hyperparameters

    batch_size = batch_size #len(x_train)
    epochs = epochs
    learning_rate = learning_rate
    loss = 0
    losses = []
    losses_val = []
    counter = 0
    

    dataset = TensorDataset( Tensor(X_train.values), torch.Tensor(Y_train['target'].values) )
    train_loader = DataLoader(dataset, batch_size = batch_size, shuffle=False)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9,0.999))
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size= 25 , gamma = 0.1)
    cross = nn.CrossEntropyLoss()
    
    #Train Model

    t = time.time()
    for epoch in range(epochs):
        for x, y in iter(train_loader):
            model.train()
            model.zero_grad()

            y_pred = model(x)
            y_val_pred = model(Tensor(X_val.values))

            loss = cross(y_pred, y.long())
            loss_val = cross(y_val_pred, Tensor(y_val['target'].values).long())

            if counter % 500 ==0:
                print('Loss after iteration {}: {}'.format(counter, loss.item()))

            losses.append(loss.item())
            losses_val.append(loss_val.item())

            loss.backward()
            optimizer.step()

            counter+=1 
        counter+=1
        scheduler.step()
    time.time()-t        


    print('Elapsed time: {} s'.format(time.time()-t))    
    print(loss.item())  
    plt.plot(range(len(losses)), losses, label = 'train')
    plt.plot(range(len(losses)), losses_val, label = 'val')
    plt.legend(loc="lower left")
    plt.show()
    
    ##Eval 

    model.eval()
    with torch.no_grad():
        val_pred = model(Tensor(X_val.values))
        train_pred = model(Tensor(X_train.values))
    y_val_pred = pd.Series(val_pred.max(1).indices).map({0:1,1:0,2:-1})
    y_train_pred = pd.Series(train_pred.max(1).indices).map({0:1,1:0,2:-1})

    print('Train accuracy: {}'.format(sklearn.metrics.accuracy_score(y_train_pred, Y_train['result'])))
    print('Val accuracy: {}'.format(sklearn.metrics.accuracy_score(y_val_pred, Y_val['result'])))

In [ ]:
train(LinearNN1, x1, y1, x_val, y_val,  5, 1024, 0.0001)
train(LinearNN2, x2, y2, x_val, y_val,  5, 1024, 0.0001)
train(LinearNN3, x3, y3, x_val, y_val,  5, 1024, 0.0001)

Loss after iteration 0: 0.9999188184738159


In [648]:
# torch.save(LinearNN, './nn0.pt')
# torch.save(LinearNN1, './nn1.pt')

In [695]:
def ensemble_predict(models, training_data, training_labels, x_val, y_val):
    ensemble = []
    val_pred_ensemble = 0
    for i,model in enumerate(models):
        StandardScaler = preprocessing.StandardScaler().fit(training_data[i].iloc[:,1:])
        X_train = pd.DataFrame(StandardScaler.transform(training_data[i].iloc[:,1:]), index = training_data[i]['match_api_id'], columns = training_data[i].iloc[:,1:].columns)
        X_val = pd.DataFrame(StandardScaler.transform(x_val.iloc[:,1:]), index = x_val['match_api_id'], columns = x_val.iloc[:,1:].columns)
        
        model.eval()
        with torch.no_grad():
            val_pred = model(Tensor(X_val.values))
            train_pred = model(Tensor(X_train.values))
        y_val_pred = pd.Series(val_pred.max(1).indices).map({0:1,1:0,2:-1})
        y_train_pred = pd.Series(train_pred.max(1).indices).map({0:1,1:0,2:-1})
        
        val_pred_ensemble += val_pred
        print('model: {}'.format(i))
        print('Train accuracy: {}'.format(sklearn.metrics.accuracy_score(y_train_pred, training_labels[i]['result'])))
        print('Val accuracy: {}'.format(sklearn.metrics.accuracy_score(y_val_pred, y_val['result'])))
        
    global y_ens
    y_ens = pd.Series(val_pred_ensemble.max(1).indices).map({0:1,1:0,2:-1})
    print('Ensemble Val accuracy: {}'.format(sklearn.metrics.accuracy_score(y_ens, y_val['result'])))


In [699]:
ensemble_predict([LinearNN1,LinearNN2,LinearNN3], [x1,x2,x3], [y1,y2,y3], x_val, y_val)

model: 0
Train accuracy: 0.5358851674641149
Val accuracy: 0.524390243902439
model: 1
Train accuracy: 0.4491366756812981
Val accuracy: 0.4413109756097561
model: 2
Train accuracy: 0.53505304763886
Val accuracy: 0.5144817073170732
Ensemble Val accuracy: 0.5274390243902439


In [697]:
ensemble_predict([LinearNN1,LinearNN3], [x1,x3], [y1,y3], x_val, y_val)

model: 0
Train accuracy: 0.5358851674641149
Val accuracy: 0.524390243902439
model: 1
Train accuracy: 0.53505304763886
Val accuracy: 0.5144817073170732
Ensemble Val accuracy: 0.5282012195121951


In [700]:
y_ens.value_counts()

 1    904
-1    408
dtype: int64